<a href="https://colab.research.google.com/github/tdizhere/AI-draft/blob/main/AI_draft1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
projjal1_human_conversation_training_data_path = kagglehub.dataset_download('projjal1/human-conversation-training-data')

print('Data source import complete.',projjal1_human_conversation_training_data_path)


100%|██████████| 41.6k/41.6k [00:00<00:00, 18.1MB/s]

Extracting files...
Data source import complete. /root/.cache/kagglehub/datasets/projjal1/human-conversation-training-data/versions/1


In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grafstor/simple-dialogs-for-chatbot")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/grafstor/simple-dialogs-for-chatbot/versions/2


In [18]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
import gc  # Garbage collection

# Set up paths
data_path = "/root/.cache/kagglehub/datasets/grafstor/simple-dialogs-for-chatbot/versions/2/dialogs.txt"

# Check for GPU availability and memory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Memory monitoring function
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

# Load tokenizer first with limited vocabulary
print("Loading tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    add_prefix_space=True,  # More efficient tokenization
    max_model_input_sizes={'gpt2': 1024}  # Limit input size
)

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Padding token set to: {tokenizer.pad_token}")

# Load dataset with memory-efficient approach
print("Loading dataset...")
class ChatDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=64, max_samples=5000):  # Reduced defaults
        self.examples = []
        self.tokenizer = tokenizer
        self.max_length = max_length

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                chat_lines = f.readlines()

            # Limit number of samples to prevent memory overload
            chat_lines = chat_lines[:max_samples*2]

            # Pair every two consecutive lines as input and target
            for i in range(0, len(chat_lines) - 1, 2):
                input_text = chat_lines[i].strip()
                target_text = chat_lines[i + 1].strip()

                # Encode inputs and targets as a single sequence
                encoded = tokenizer(
                    f"{input_text} {tokenizer.eos_token} {target_text}",
                    max_length=max_length,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt"
                )

                self.examples.append({
                    "input_ids": encoded["input_ids"].squeeze(),
                    "attention_mask": encoded["attention_mask"].squeeze(),
                })

        except FileNotFoundError:
            print(f"Error: File not found at {file_path}")
            raise
        except Exception as e:
            print(f"Error loading dataset: {e}")
            raise

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Initialize dataset with smaller batch size and fewer samples
dataset = ChatDataset(data_path, tokenizer, max_length=64, max_samples=5000)

# DataLoader with reduced batch size and workers
batch_size = 4  # Reduced batch size
data_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,  # Use multiple workers if possible
    pin_memory=True  # Faster data transfer to GPU
)

# Clear GPU memory before model loading
torch.cuda.empty_cache()
gc.collect()

print_gpu_memory()

# Load model with lower precision to save memory
model = GPT2LMHeadModel.from_pretrained("gpt2", torch_dtype=torch.float16)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

print_gpu_memory()

# Optimizer with lower learning rate
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# Training loop with safety checks
epochs = 3
print("Starting training...")
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    # Clear memory before each epoch
    torch.cuda.empty_cache()
    gc.collect()

    for batch_idx, batch in enumerate(data_loader):
        try:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # Clear gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids,
            )
            loss = outputs.loss
            epoch_loss += loss.item()

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Print progress and clear memory periodically
            if batch_idx % 10 == 0:
                print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item()}")
                torch.cuda.empty_cache()
                gc.collect()

        except RuntimeError as e:
            print(f"CUDA out of memory error: {e}")
            # Clear memory and try to continue
            torch.cuda.empty_cache()
            gc.collect()
            continue

    print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {epoch_loss / len(data_loader)}")

# Save model
model_save_path = "/conversational_model"
os.makedirs(model_save_path, exist_ok=True)
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model saved at {model_save_path}")

Using device: cuda
Loading tokenizer...
Padding token set to: <|endoftext|>
Loading dataset...
Allocated: 1.08 GB
Cached: 1.18 GB
Allocated: 1.35 GB
Cached: 1.38 GB
Starting training...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Batch 0, Loss: 6.44921875
Epoch 1, Batch 10, Loss: 2.265625
Epoch 1, Batch 20, Loss: 2.236328125
Epoch 1, Batch 30, Loss: 1.4580078125
Epoch 1, Batch 40, Loss: 1.8828125
Epoch 1, Batch 50, Loss: 1.4794921875
Epoch 1, Batch 60, Loss: 1.48046875
Epoch 1, Batch 70, Loss: 1.7333984375
Epoch 1, Batch 80, Loss: 1.44921875
Epoch 1, Batch 90, Loss: 1.38671875
Epoch 1, Batch 100, Loss: 1.2646484375
Epoch 1, Batch 110, Loss: 1.9501953125
Epoch 1, Batch 120, Loss: 1.2958984375
Epoch 1, Batch 130, Loss: 1.443359375
Epoch 1, Batch 140, Loss: 1.509765625
Epoch 1, Batch 150, Loss: 1.8916015625
Epoch 1, Batch 160, Loss: 1.576171875
Epoch 1, Batch 170, Loss: 1.2646484375
Epoch 1, Batch 180, Loss: 1.244140625
Epoch 1, Batch 190, Loss: 1.921875
Epoch 1, Batch 200, Loss: 1.4267578125
Epoch 1, Batch 210, Loss: 1.55078125
Epoch 1, Batch 220, Loss: 1.5556640625
Epoch 1, Batch 230, Loss: 1.0390625
Epoch 1, Batch 240, Loss: 1.9970703125
Epoch 1, Batch 250, Loss: 1.7119140625
Epoch 1, Batch 260, Loss: 

In [4]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import gc
import json

# Configuration
CONFIG_PATH = "/conversational_model/training_config.json"

# Utility function to save training state
def save_checkpoint(model, optimizer, scheduler, epoch, loss, checkpoint_path):
    """Save model checkpoint with training state."""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")

# Utility function to load checkpoint
def load_checkpoint(checkpoint_path, model, optimizer, scheduler, device):
    """Load model checkpoint and training state."""
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return checkpoint['epoch'], checkpoint['loss']

# Enhanced Dataset with more robust loading
class ChatDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=128, max_samples=None):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_length = max_length

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                chat_lines = f.readlines()

            # Optional sample limit
            if max_samples:
                chat_lines = chat_lines[:max_samples*2]

            # Process lines into training examples
            for i in range(0, len(chat_lines) - 1, 2):
                input_text = chat_lines[i].strip()
                target_text = chat_lines[i + 1].strip()

                # Combine input and target with special token
                combined_text = f"{input_text} {tokenizer.eos_token} {target_text}"

                # Tokenize
                encoded = tokenizer(
                    combined_text,
                    max_length=max_length,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt"
                )

                self.examples.append({
                    "input_ids": encoded["input_ids"].squeeze(),
                    "attention_mask": encoded["attention_mask"].squeeze(),
                })

        except Exception as e:
            print(f"Error loading dataset: {e}")
            raise

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Model Training Function
def train_model(
    data_path,
    model_save_dir='/conversational_model',
    resume_training=False,
    max_epochs=5,
    batch_size=4
):
    # Ensure save directories exist
    os.makedirs(model_save_dir, exist_ok=True)
    checkpoint_dir = os.path.join(model_save_dir, 'checkpoints')
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Device setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Tokenizer setup
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token

    # Dataset preparation
    full_dataset = ChatDataset(data_path, tokenizer, max_length=128, max_samples=10000)

    # Split dataset
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Model setup
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    model.resize_token_embeddings(len(tokenizer))
    model = model.to(device)

    # Optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=5e-5)
    total_steps = len(train_loader) * max_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1*total_steps),
        num_training_steps=total_steps
    )

    # Check for existing checkpoint
    start_epoch = 0
    best_val_loss = float('inf')

    if resume_training:
        checkpoint_files = sorted(
            [f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint_')],
            key=lambda x: int(x.split('_')[1].split('.')[0])
        )

        if checkpoint_files:
            latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[-1])
            start_epoch, best_val_loss = load_checkpoint(
                latest_checkpoint, model, optimizer, scheduler, device
            )
            print(f"Resuming training from epoch {start_epoch}")

    # Training loop
    for epoch in range(start_epoch, max_epochs):
        model.train()
        train_losses = []

        for batch_idx, batch in enumerate(train_loader):
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids
            )

            loss = outputs.loss
            loss.backward()

            train_losses.append(loss.item())

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            # Periodic checkpoint
            if batch_idx % 100 == 0:
                avg_train_loss = np.mean(train_losses)
                print(f"Epoch {epoch+1}, Batch {batch_idx}, Train Loss: {avg_train_loss}")

        # Validation
        model.eval()
        val_losses = []
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=input_ids
                )
                val_losses.append(outputs.loss.item())

        avg_val_loss = np.mean(val_losses)
        avg_train_loss = np.mean(train_losses)

        print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss}, Validation Loss = {avg_val_loss}")

        # Save checkpoint
        checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_{epoch+1}.pt')
        save_checkpoint(model, optimizer, scheduler, epoch+1, avg_val_loss, checkpoint_path)

        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_path = os.path.join(model_save_dir, 'best_model')
            model.save_pretrained(best_model_path)
            tokenizer.save_pretrained(best_model_path)
            print(f"Best model saved with validation loss: {best_val_loss}")

    # Final model save
    model.save_pretrained(model_save_dir)
    tokenizer.save_pretrained(model_save_dir)
    print("Training completed and model saved.")

    return model, tokenizer

# Inference Function
def chat_with_model(model, tokenizer, prompt, max_length=100, num_return_sequences=1, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model.eval()

    # Tokenize input
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Generate response
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

    # Decode responses
    responses = [tokenizer.decode(seq, skip_special_tokens=True) for seq in output]
    return responses

# Testing Function
def test_model(model, tokenizer, test_prompts):
    print("\n--- Model Testing ---")
    for prompt in test_prompts:
        print(f"\nPrompt: {prompt}")
        responses = chat_with_model(model, tokenizer, prompt)
        for i, response in enumerate(responses, 1):
            print(f"Response {i}: {response}")

# Main Execution
def main():
    # Set paths
    data_path = "/root/.cache/kagglehub/datasets/projjal1/human-conversation-training-data/versions/1/human_chat.txt"
    model_save_dir = "/conversational_model"

    # Train model
    trained_model, trained_tokenizer = train_model(
        data_path,
        model_save_dir,
        resume_training=True,  # Will resume from last checkpoint if exists
        max_epochs=5,
        batch_size=4
    )

    # Test prompts
    test_prompts = [
        "Hello, how are you?",
        "What is the meaning of life?",
        "Tell me a short story.",
        "Explain quantum computing in simple terms."
    ]

    # Test the model
    test_model(trained_model, trained_tokenizer, test_prompts)

if __name__ == "__main__":
    main()

Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Batch 0, Train Loss: 7.369625568389893
Epoch 1, Batch 100, Train Loss: 2.2544064152948926
Epoch 1: Train Loss = 1.825354473789533, Validation Loss = 0.9632481885583777
Checkpoint saved at /conversational_model/checkpoints/checkpoint_1.pt
Best model saved with validation loss: 0.9632481885583777
Epoch 2, Batch 0, Train Loss: 0.6830458045005798
Epoch 2, Batch 100, Train Loss: 0.8678857468732513
Epoch 2: Train Loss = 0.8804653787612915, Validation Loss = 0.9628808631708747
Checkpoint saved at /conversational_model/checkpoints/checkpoint_2.pt
Best model saved with validation loss: 0.9628808631708747
Epoch 3, Batch 0, Train Loss: 0.9417645335197449
Epoch 3, Batch 100, Train Loss: 0.8259283732069601
Epoch 3: Train Loss = 0.7895452650388082, Validation Loss = 0.9824455090259251
Checkpoint saved at /conversational_model/checkpoints/checkpoint_3.pt
Epoch 4, Batch 0, Train Loss: 0.4639434814453125
Epoch 4, Batch 100, Train Loss: 0.7235138889881644
Epoch 4: Train Loss = 0.735599592228730

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpec

Training completed and model saved.

--- Model Testing ---

Prompt: Hello, how are you?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Response 1: Hello, how are you? 

Prompt: What is the meaning of life?
Response 1: What is the meaning of life? 

Prompt: Tell me a short story.
Response 1: Tell me a short story. I'm a comic book artist. 

Prompt: Explain quantum computing in simple terms.
Response 1: Explain quantum computing in simple terms. It's a very exciting idea, and I'm excited to try it out. 


In [14]:
git clone https://huggingface.co/datasets/flammenai/casual-conversation-DPO

SyntaxError: invalid syntax (<ipython-input-14-d687f01b680a>, line 1)

# ***`Training`***

In [16]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
import gc
from torch.cuda.amp import autocast, GradScaler

# Set up paths
data_path = "/root/.cache/kagglehub/datasets/grafstor/simple-dialogs-for-chatbot/versions/2/dialog.txt"

# Check for GPU availability and memory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Memory monitoring function
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

# Load tokenizer first with limited vocabulary
print("Loading tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2",
    add_prefix_space=True,
    max_model_input_sizes={'gpt2': 1024}
)

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Padding token set to: {tokenizer.pad_token}")

# Load dataset with memory-efficient approach
print("Loading dataset...")
class ChatDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_length=64, max_samples=5000):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_length = max_length

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                chat_lines = f.readlines()

            # Limit number of samples to prevent memory overload
            chat_lines = chat_lines[:max_samples*2]

            # Pair every two consecutive lines as input and target
            for i in range(0, len(chat_lines) - 1, 2):
                input_text = chat_lines[i].strip()
                target_text = chat_lines[i + 1].strip()

                # Encode inputs and targets as a single sequence
                encoded = tokenizer(
                    f"{input_text} {tokenizer.eos_token} {target_text}",
                    max_length=max_length,
                    truncation=True,
                    padding="max_length",
                    return_tensors="pt"
                )

                self.examples.append({
                    "input_ids": encoded["input_ids"].squeeze(),
                    "attention_mask": encoded["attention_mask"].squeeze(),
                })

        except FileNotFoundError:
            print(f"Error: File not found at {file_path}")
            raise
        except Exception as e:
            print(f"Error loading dataset: {e}")
            raise

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Initialize dataset with smaller batch size and fewer samples
dataset = ChatDataset(data_path, tokenizer, max_length=64, max_samples=5000)

# DataLoader with optimized batch size and workers
batch_size = 8  # Increase batch size if GPU allows
data_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,  # More workers for faster data loading
    pin_memory=True  # Faster data transfer to GPU
)

# Clear GPU memory before model loading
torch.cuda.empty_cache()
gc.collect()

print_gpu_memory()

# Load model with mixed precision (float16)
model = GPT2LMHeadModel.from_pretrained("gpt2", torch_dtype=torch.float16)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

print_gpu_memory()

# Optimizer with lower learning rate
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# Initialize GradScaler for mixed precision
scaler = GradScaler()

# Training loop with gradient accumulation and mixed precision
epochs = 3
print("Starting training...")
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    # Clear memory before each epoch
    torch.cuda.empty_cache()
    gc.collect()

    for batch_idx, batch in enumerate(data_loader):
        try:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # Clear gradients
            optimizer.zero_grad()

            # Mixed precision forward pass
            with autocast():
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=input_ids,
                )
                loss = outputs.loss

            # Scale the loss for mixed precision
            scaler.scale(loss).backward()

            # Step the optimizer and scaler
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()

            # Print progress and clear memory periodically
            if batch_idx % 10 == 0:
                print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item()}")
                torch.cuda.empty_cache()
                gc.collect()

        except RuntimeError as e:
            print(f"CUDA out of memory error: {e}")
            # Clear memory and try to continue
            torch.cuda.empty_cache()
            gc.collect()
            continue

    print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {epoch_loss / len(data_loader)}")

# Save model
model_save_path = "/conversational_model"
os.makedirs(model_save_path, exist_ok=True)
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model saved at {model_save_path}")


Using device: cuda
Loading tokenizer...
Padding token set to: <|endoftext|>
Loading dataset...
Error: File not found at /root/.cache/kagglehub/datasets/grafstor/simple-dialogs-for-chatbot/versions/2/dialog.txt


FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/kagglehub/datasets/grafstor/simple-dialogs-for-chatbot/versions/2/dialog.txt'

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

class ModelChatbot:
    def __init__(self, model_path):
        """
        Initialize the chatbot with a pre-trained model

        Args:
            model_path (str): Path to the saved model directory
        """
        # Determine device
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Load model and tokenizer
        try:
            self.model = GPT2LMHeadModel.from_pretrained(model_path)
            self.tokenizer = GPT2Tokenizer.from_pretrained(model_path)

            # Ensure pad token is set
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token

            # Move model to device
            self.model = self.model.to(self.device)
            self.model.eval()  # Set to evaluation mode

            print("Model loaded successfully!")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise

    def generate_response(self, prompt, max_length=100, temperature=0.7, top_k=50, top_p=0.95):
        """
        Generate a response to the given prompt

        Args:
            prompt (str): Input text to generate response for
            max_length (int): Maximum length of generated text
            temperature (float): Controls randomness of predictions
            top_k (int): Number of highest probability tokens to keep
            top_p (float): Nucleus sampling probability threshold

        Returns:
            str: Generated response
        """
        try:
            # Tokenize input
            input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(self.device)

            # Generate response
            with torch.no_grad():
                output = self.model.generate(
                    input_ids,
                    max_length=max_length,
                    num_return_sequences=1,
                    no_repeat_ngram_size=2,
                    temperature=temperature,
                    top_k=top_k,
                    top_p=top_p,
                )

            # Decode response
            response = self.tokenizer.decode(output[0], skip_special_tokens=True)

            # Remove the original prompt from the response
            response = response[len(prompt):].strip()

            return response

        except Exception as e:
            print(f"Error generating response: {e}")
            return "I'm having trouble generating a response."

    def interactive_chat(self):
        """
        Start an interactive chat session with the model
        """
        print("🤖 AI Chatbot Initialized! Type 'quit' to exit.")
        print("-------------------------------------------")

        # Chat loop
        while True:
            try:
                # Get user input
                user_input = input("You: ").strip()

                # Check for exit command
                if user_input.lower() in ['quit', 'exit', 'bye']:
                    print("Goodbye! 👋")
                    break

                # Generate and print response
                if user_input:
                    response = self.generate_response(user_input)
                    print("AI: " + response)

            except KeyboardInterrupt:
                print("\nChat interrupted. Type 'quit' to exit.")
            except Exception as e:
                print(f"An error occurred: {e}")

def main():
    # Path to the saved model
    model_path = "/conversational_model"  # Adjust this to your model's path

    try:
        # Create chatbot instance
        chatbot = ModelChatbot(model_path)

        # Start interactive chat
        chatbot.interactive_chat()

    except Exception as e:
        print(f"Failed to initialize chatbot: {e}")

if __name__ == "__main__":
    main()

Using device: cuda
Model loaded successfully!
🤖 AI Chatbot Initialized! Type 'quit' to exit.
-------------------------------------------
You: hii


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AI: i.	i'm sorry for you.
You: how are you


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AI: u going to go?	i'm going.
You: dont go please


...

AI: ?	i'm having a party next saturday.
You: am i invited?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AI: ?	i'm sorry for you.
You: i love you


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AI: u to go to hell.	i'll go back upstairs anyway.
